In [2]:
from __future__ import annotations
from pydantic import BaseModel, Field


from langchain.agents import (
    AgentExecutor,
    create_openai_tools_agent,
)

from langgraph.checkpoint.memory import MemorySaver

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)

from GoogleCalender import CreateGoogleCalendarEvent
from langchain_community.tools.gmail.utils import (
    build_resource_service,
    get_gmail_credentials,
)

memory = MemorySaver()

In [3]:
class MeetingDetils(BaseModel):
    start_data_time: str
    end_date_time: str
    agenda: str
    user_email: str

In [4]:
class GetEventsSchema(BaseModel):
    start_datetime: str = Field(
        description=(
            "The start datetime for the event in the following format: "
            'YYYY-MM-DDTHH:MM:SS, where "T" separates the date and time.'
        )
    )
    end_datetime: str = Field(
        description=(
            "The end datetime for the event in the following format: "
            'YYYY-MM-DDTHH:MM:SS, where "T" separates the date and time.'
        )
    )
    agenda: str
    user_email: str

    timezone: str = Field(
        default="Asia/Kolkata",
        description="The timezone in TZ Database Name format, e.g. 'America/New_York'",
    )

In [5]:
@tool
def identity(query):
    """This is the identity function return whatever user pass in query."""
    return query

In [6]:
credentials = get_gmail_credentials(
    token_file="../token.json",
    scopes=["https://www.googleapis.com/auth/calendar"],
    client_secrets_file="credentials.json",
)

calendar_service = build_resource_service(
    credentials=credentials, service_name="calendar", service_version="v3"
)

createeventtool = CreateGoogleCalendarEvent.from_api_resource(calendar_service)

In [11]:
llm = ChatOpenAI()

In [7]:
def create_agent(llm, tools: list, system_prompt):

    agent = create_openai_tools_agent(llm, tools, system_prompt)
    executor = AgentExecutor(agent=agent, tools=tools)

    return executor

In [14]:
demo_prompt = """You are responsible for managing user queries related to demo requests. Your primary role is to gather all necessary details from the user, and then convert that information into the required format as outlined in {format_instructions}. Do not make any assumptions; simply collect the information provided by the user.

Steps:
Take the details from user. and you have to scadule a meeting for it in a google calender for that you have the access to following tools:
{tools}
Express your appreciation for the user's cooperation with date and time of the event. Example: "Thanks for providing the details! We'll arrange the demo on 25th august at 10:30 am to 11:30 am."
End of Task
"""

demo_parser = JsonOutputParser(pydantic_object=GetEventsSchema)

In [15]:
demo_prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            demo_prompt,
        ),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ],
).partial(
    format_instructions=demo_parser.get_format_instructions(), tools=[createeventtool]
)

In [16]:
demo = create_agent(
    llm=llm, tools=[createeventtool], system_prompt=demo_prompt_template
)

In [19]:
demo.invoke(
    {
        "messages": [         
            HumanMessage(
                content="i want demo at 10:30 am to 11:30 am 20th september 2024 and agenda is presentation and my mail ID is rohanbanda103@gmail.com",
            )
        ]
    }
)

{'messages': [HumanMessage(content='i want demo at 10:30 am to 11:30 am 20th september 2024 and agenda is presentation and my mail ID is rohanbanda103@gmail.com')],
 'output': "Thanks for providing the details! We'll arrange the demo on 20th September 2024 at 10:30 am to 11:30 am."}